In [1]:
import sys, os, time
%load_ext autoreload
%autoreload 2
import torch
import torch.nn.functional as F
from torch.autograd import grad
import torch.distributions as distrib
import torch.nn as nn
import torch.utils.data as td
from torch.utils.data import Dataset, DataLoader
import gym
import numpy as np
#%matplotlib notebook
import matplotlib.pyplot as plt
plt.switch_backend('TKAgg') #('Qt5Agg')
from scipy.sparse import coo_matrix
#from envs import Discrete_Pos_Trap, Gym_Env, Print_Env
from torch.distributions.utils import lazy_property
import foundation as fd
import foundation.util as util
from foundation.util import NS
print('prep done')

prep done


In [2]:
args = NS()
args.name = 'test'

args.txtlog = False
args.tblog = True
args.save_root = 'results'
args.num_workers = 2

args.def_type = 'torch.FloatTensor'

#args.env = 'cartpole'
args.env = 'pendulum'
args.base_seed = 12

args.reg_coeff = 1e-6
args.time_order = 3
args.obs_order = 1
args.max_steps = 10
args.nonlinearity = 'prelu'
args.hidden = [6,6]

args.delta = 0.01
args.gamma = 0.99

args.num_iter = 100
args.num_traj = 20
args.num_eval = 5

print('Name: {}'.format(args.name))

Name: test


In [8]:
now = time.strftime("%y-%m-%d-%H%M%S")
args.save_dir = os.path.join(args.save_root, args.name, now)
#args.save_dir = os.path.join(args.save_root, args.name)
print('Save dir: {}'.format(args.save_dir))
if args.tblog or args.txtlog:
    util.create_dir(args.save_dir)
    print('Logging in {}'.format(args.save_dir))
logger = util.Logger(args.save_dir, tensorboard=args.tblog, txt=args.txtlog)
env = util.get_env(args.env)
obs_dim = env.spec.obs_space.size
act_dim = env.spec.action_space.choices if args.env == 'cartpole' else env.spec.action_space.size
print('Env: {} (obs={},act={})'.format(args.env, obs_dim, act_dim))

Save dir: results/test/18-11-02-135413
Logging in results/test/18-11-02-135413
Env: pendulum (obs=3,act=1)


In [9]:
model = fd.nets.MLP(input_dim=obs_dim, output_dim=act_dim, hidden_dims=args.hidden, nonlinearity=args.nonlinearity)
print(model)
policy = fd.policies.Gaussian_Policy(model=model, def_type=args.def_type, init_log_std=0, min_log_std=-3)
#policy = fd.policies.Categorical_Policy(model=model, def_type=args.def_type)
baseline = fd.baselines.LinearBaselineLeastSqrs(env.spec.obs_space.size)

agent = fd.agents.NPG(policy, baseline=baseline, )

total_episodes = 0

MLP(
  (net): Sequential(
    (0): Linear(in_features=3, out_features=6, bias=True)
    (1): PReLU(num_parameters=1)
    (2): Linear(in_features=6, out_features=6, bias=True)
    (3): PReLU(num_parameters=1)
    (4): Linear(in_features=6, out_features=1, bias=True)
  )
)


In [10]:
loader = fd.data.GeneratorLoader(policy=policy, use_transition=False, batch_size=args.num_traj, num_workers=args.num_workers, 
                                 env_name=args.env, env_kwargs={}, episode_budget=args.num_traj*args.num_iter, 
                                 seed=args.base_seed)

In [11]:
seed = args.base_seed
score = None
stats = util.StatsMeter()
time_stats = util.StatsMeter('traj', 'baseline', 'train', 'eval')
best_perf = None
start = time.time()
for itr, paths in enumerate(loader):
    
    time_stats.update('traj', time.time() - start)
    start = time.time()
    
    train_stats = agent.train_step(paths)
    stats.join(train_stats, prefix='train-')
    
    #break
    
    time_stats.update('train', time.time() - start)
    start = time.time()
    
    # eval
    eval_stats = loader.evaluate_policy(args.num_eval)
    stats.join(eval_stats, prefix='eval-')
    time_stats.update('eval', time.time() - start)
    
    if best_perf is None or stats['eval-score'].avg >= best_perf:
        best_perf = stats['eval-score'].avg
    print("[ {} ] {:}/{:} (N={}) : {:5.3f} {:5.3f} {:5.3} ".format((time.strftime("%b-%d-%y %H:%M:%S")),itr+1, args.num_iter, len(loader),
                                                                   stats['train-returns'].avg, stats['eval-score'].avg, best_perf if best_perf is not None else ''))
    logger.update(stats.avgs())
    logger.update({'time-'+k:v for k,v in time_stats.vals().items()})
    #print(list(policy.parameters())[-1])
    # save model
    start = time.time()
    

[ Nov-02-18 13:54:27 ] 1/100 (N=4000) : -409.913 -1580.660 -1.58e+03 
[ Nov-02-18 13:54:38 ] 2/100 (N=8000) : -437.279 -1364.361 -1.36e+03 
[ Nov-02-18 13:54:51 ] 3/100 (N=12000) : -401.045 -1317.009 -1.32e+03 
[ Nov-02-18 13:55:09 ] 4/100 (N=16000) : -385.899 -1304.751 -1.3e+03 
[ Nov-02-18 13:55:20 ] 5/100 (N=20000) : -382.498 -1276.362 -1.28e+03 
[ Nov-02-18 13:55:31 ] 6/100 (N=24000) : -373.348 -1292.382 -1.28e+03 
[ Nov-02-18 13:55:41 ] 7/100 (N=28000) : -375.717 -1320.620 -1.28e+03 
[ Nov-02-18 13:55:53 ] 8/100 (N=32000) : -380.131 -1347.709 -1.28e+03 
[ Nov-02-18 13:56:04 ] 9/100 (N=36000) : -384.321 -1358.215 -1.28e+03 
[ Nov-02-18 13:56:17 ] 10/100 (N=40000) : -385.144 -1359.279 -1.28e+03 
[ Nov-02-18 13:56:28 ] 11/100 (N=44000) : -393.844 -1352.496 -1.28e+03 
[ Nov-02-18 13:56:39 ] 12/100 (N=48000) : -400.922 -1352.635 -1.28e+03 


Process Process-5:
Process Process-6:
Traceback (most recent call last):
  File "/home/fleeb/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/fleeb/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/mnt/c/Users/anwan/OneDrive - UW Office 365/Khan/AIResearch/foundation/foundation/util/farming.py", line 28, in _worker_loop
    args = in_queue.get()
  File "/home/fleeb/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/fleeb/anaconda3/lib/python3.6/multiprocessing/queues.py", line 94, in get
    res = self._recv_bytes()
  File "/home/fleeb/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/fleeb/anaconda3/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_b

KeyboardInterrupt: 

In [ ]:
args.num_workers

In [9]:
train_stats

StatsMeter(objective-delta:0.0025, mean-kl:0.0102, score:12.6551, returns:10.5787, alpha:7.9396, vpg-norm:0.0141, npg-norm:0.1844, bsln-error-before:50.0865, bsln-error-final:25.0193)